In [1]:
import os
import csv
import pandas as pd
import datetime

DATA_PATH = "data"

In [2]:
inns = pd.read_excel(os.path.join(DATA_PATH, "список инн.xlsx"), dtype={"requested_inn_dadata": str})
print("Len inns - ", inns.shape[0])
# inns = inns[inns.requested_inn_dadata.apply(lambda s: len(s) in [10, 12])]
# print("Len inns after filter - ", inns.shape[0])

Len inns -  9080


In [3]:
inns_info = pd.read_csv(os.path.join(DATA_PATH, "out_file.csv"), engine="c", encoding="utf-16", sep='\t', error_bad_lines=False, dtype={'INN': object})

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
inns_info.columns

Index(['Закупки по', 'Реестровый номер закупки',
       'Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)',
       'Наименование закупки',
       'Предмет электронного аукциона (только для ПП РФ 615)',
       'Идентификационный код закупки', 'Номер лота', 'Наименование лота',
       'Начальная (максимальная) цена контракта', 'Валюта',
       'Начальная (максимальная) цена в валюте контракта', 'Валюта контракта',
       'Классификация по ОКДП', 'Классификация по ОКПД',
       'Классификация по ОКПД2', 'Код позиции', 'Наименование Заказчика',
       'Организация, осуществляющая размещение ', 'Дата размещения',
       'Дата обновления', 'Этап закупки',
       'Особенности размещения заказа\ Преимущества (44-ФЗ)',
       'Дата начала подачи заявок', 'Дата окончания подачи заявок',
       'Дата проведения электронного аукциона', 'INN'],
      dtype='object')

In [11]:
def convert_currency(curname, amount):
    dict_curr_course = {"USD": 73, "EUR": 87, "RUB": 1}
    return amount * dict_curr_course[curname] if amount is not None else None

def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

In [12]:
inns_info.INN.apply(lambda s: len(s)).value_counts()

10    180189
12     30676
9       7977
11      2201
3       1512
Name: INN, dtype: int64

In [13]:
#inns_info.dropna(subset=['INN'], inplace=True)
#inns_info.dropna(subset=['Дата размещения'], inplace=True)
inns_info.INN = inns_info.INN.apply(lambda s: '0' + s if len(s) in [9, 11] else s)
assert set(inns_info.INN.apply(lambda s: len(s)).value_counts().index) == {3, 10, 12}
inns_info["Дата размещения"] = pd.to_datetime(inns_info["Дата размещения"])
inns_info = inns_info[inns_info["Валюта"].isin(["RUB", "USD", "EUR"])]
inns_uniq = inns_info.INN.unique()
inns_info["Начальная (максимальная) цена контракта"] = inns_info.apply(lambda x: convert_currency(x[9], x[8]), axis=1)
log = inns_info["Начальная (максимальная) цена контракта"].astype('str').apply(isfloat)
inns_info = inns_info[log]
inns_info["Начальная (максимальная) цена контракта"] = inns_info["Начальная (максимальная) цена контракта"].astype("float")
inns_info["year"] = inns_info["Дата размещения"].apply(lambda x: x.year)

In [14]:
print("Len inns in csv - ", len(set(inns_uniq)))
print("Len inns in excel - ", len(set(inns.requested_inn_dadata.unique())))
print("Len intersection - ", len(set(inns_uniq).intersection(inns.requested_inn_dadata.unique())))

Len inns in csv -  3038
Len inns in excel -  9079
Len intersection -  3038


In [15]:
sum_price = inns_info.groupby("INN")["Начальная (максимальная) цена контракта"].sum().to_dict()

In [16]:
max_price = inns_info.groupby("INN")["Начальная (максимальная) цена контракта"].max().to_dict()

In [17]:
min_price = inns_info.groupby("INN")["Начальная (максимальная) цена контракта"].min().to_dict()

In [18]:
mean_price = inns_info.groupby("INN")["Начальная (максимальная) цена контракта"].mean().to_dict()

In [19]:
count_price = inns_info.groupby("INN")["Начальная (максимальная) цена контракта"].count().to_dict()

In [20]:
first_date = inns_info.groupby("INN")["Дата размещения"].min().to_dict()

In [21]:
last_date = inns_info.groupby("INN")["Дата размещения"].max().to_dict()

In [22]:
df = pd.DataFrame({"inn": inns_uniq})
df.inn = df.inn.astype('str')

In [23]:
df["sum_price"] = df.inn.apply(lambda x: sum_price[x] if x in sum_price else None)
df["max_price"] = df.inn.apply(lambda x: max_price[x] if x in max_price else None)
df["min_price"] = df.inn.apply(lambda x: min_price[x] if x in min_price else None)
df["mean_price"] = df.inn.apply(lambda x: mean_price[x] if x in mean_price else None)
df["count_price"] = df.inn.apply(lambda x: count_price[x] if x in count_price else None)
df.dropna(subset=["max_price"], inplace=True)
df["count_price"] = df["count_price"].astype('int')
df["first_date"] = df.inn.apply(lambda x: first_date[x] if x in first_date else None)
df["last_date"] = df.inn.apply(lambda x: last_date[x] if x in last_date else None)

In [24]:
for year in list(range(2011, 2022)):
    df["count_{0}".format(year)] = 0

In [25]:
for year in list(range(2011, 2022)):
    map_year = inns_info.groupby("INN").year.apply(lambda x: (x == year).sum()).to_dict()
    df["count_{0}".format(year)] = df.inn.apply(lambda x: map_year[x] if x in map_year else None)

In [26]:
for kind in (inns_info["Закупки по"].value_counts()).index:
    mapper = inns_info.groupby("INN")["Закупки по"].apply(lambda x: (x == kind).sum()).to_dict()
    df["count_{0}".format(kind)] = df.inn.apply(lambda x: mapper[x] if x in mapper else None)

In [27]:
df.to_csv(os.path.join(DATA_PATH, "features.csv"), index=False)

In [30]:
df = pd.read_csv(os.path.join(DATA_PATH, "features.csv"))

In [22]:
df.inn

0       100000000000
1         1001270298
2         1001289010
3         1001294940
4         1001345722
            ...     
3033      9731010674
3034      9731044458
3035    973301605244
3036      9999999999
3037    999999999999
Name: inn, Length: 3038, dtype: object

### Tf Idf transformation

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [28]:
#Preprocess function
def preprocess_text(text):
    #tokens = mystem.lemmatize(text.lower())
    tokens = text.lower().split()
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [29]:
def get_n_words(corpus, n):
    prep_corpus = []
    for text in corpus:
        prep_corpus.append(preprocess_text(text))
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(prep_corpus)
    M = X.toarray()
    words = []
    i = 0
    for ind in M.sum(axis=0).argsort()[::-1]:
        if vectorizer.get_feature_names()[ind] in stop_words:
            continue
        else:
            words.append(vectorizer.get_feature_names()[ind])
            i += 1
        if i == 5:
            break
    return ','.join(words)

In [30]:
inn_to_words = inns_info.groupby("INN")["Наименование закупки"].apply(lambda corpus: get_n_words(list(corpus), 3)).to_dict()

NameError: name 'stop_words' is not defined

In [75]:
df["words"] = df.inn.apply(lambda x: inn_to_words[x] if x in inn_to_words else None)

In [77]:
df.words[:50]

0     предоставлению,спектаклях,участия,нгатоиб,дрес...
1            км,моста,дороги,автомобильной,документации
2     покрытия,асфальтобетонного,территории,дорог,ас...
3     материалов,строительных,оборудования,материалы...
4                сносу,многоквартирных,домов,жилых,кеми
5      оконных,отопления,промывке,зданий,электрочайники
6             товаров,канцелярских,бумага,писчая,кресел
7           строительных,монтажных,здания,филиалов,нэск
8        капитальный,адыгея,здания,района,строительство
9             оконных,замена,блоков,капитальный,текущий
10        пензы,травы,территории,благоустройству,района
11     светильников,светодиодных,электротоваров,ламп,рк
12               территории,сыктывкар,контейнеров,го,мо
13                             мяса,птицы,кур,года,2018
14               здания,дэс,сыктывкар,документации,коми
15    хозяйство,технических,текущий,сыктывкар,помещений
16                      помещений,рк,сыктывкар,гбу,коми
17                       ухта,пгт,сносу,дома,тер

In [31]:
all_texts = inns_info["Наименование закупки"]
prep_all_texts = []
for text in all_texts:
    prep_all_texts.append(preprocess_text(text))

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(prep_all_texts)

In [ ]:
stop_words = []
for ind in np.squeeze(np.asarray(X.sum(axis=0))).argsort()[-10:][::-1]:
    stop_words.append(vectorizer.get_feature_names()[ind])